In [107]:
import pandas as pd
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
sw=stopwords.words('english')
ps=PorterStemmer()
import json 

In [127]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
	SELECT ID, QGROUP, QPK, MAX_Q_PK, QOFFSET, MAX_QG_PK, QGOFFSET, MAX_FK_Form,QTEXT 
	FROM vw_MetricsCompositeKeys WHERE PK_Form = '2020-A-SAOP'   
""" 
df = pd.read_sql(query,con=conn)   
df.to_excel(r'C:\_som\_src\_compile\saop\SAOP.xlsx','SAOP') 
df 

,ID,QGROUP,QPK,MAX_Q_PK,QOFFSET,MAX_QG_PK,QGOFFSET,MAX_FK_Form,QTEXT
0,1-0-,2150,21200,23813,0,2617,0,2517,"Note:<BR />If ""No"" is selected for metric 1a, ..."
1,1-1-1a,2150,21201,23813,1,2617,0,2517,Has the head of the agency designated a Senior...
2,1-2-1b,2150,21202,23813,2,2617,0,2517,"Has the agency reported the name, title, and c..."
3,1-3-1c,2150,21203,23813,3,2617,0,2517,"Does the SAOP have the necessary position, exp..."
4,1-4-1d,2150,21204,23813,4,2617,0,2517,Does the SAOP have the necessary role and resp...
...,...,...,...,...,...,...,...,...,...
92,17-7-14g,2166,21402,23813,202,2617,16,2517,Does the agency's Privacy Program Page include...
93,17-9-14h,2166,21403,23813,203,2617,16,2517,Does the agency's Privacy Program Page include...
94,17-10-14i,2166,21404,23813,204,2617,16,2517,Does the agency's Privacy Program Page include...
95,17-11-14j,2166,21405,23813,205,2617,16,2517,Does the agency's Privacy Program Page include...


In [124]:
QGR = 3000
d = {} 
for qg in df.QGROUP.unique():
    QGR+=1
    d[str(qg)]= str(QGR)
d    

{'2150': '3001',
 '2151': '3002',
 '2152': '3003',
 '2153': '3004',
 '2154': '3005',
 '2155': '3006',
 '2156': '3007',
 '2157': '3008',
 '2158': '3009',
 '2159': '3010',
 '2160': '3011',
 '2161': '3012',
 '2162': '3013',
 '2163': '3014',
 '2164': '3015',
 '2165': '3016',
 '2166': '3017'}

In [125]:
QGOFFSET = -1
QPK = 24500 
for i,r in df.iterrows():      
    if QGOFFSET != r.QGOFFSET:
        QGOFFSET = r.QGOFFSET
        QPK += 25
    QPK+=1
    d[str(r.QPK)]= str(QPK)  

In [122]:
with open("C:\_som\_src\_compile\saop\gapkeyval.json", "w") as f: 
    json.dump(d, f, indent=4)

In [60]:
conn.close()

In [145]:
t = """
Has the as Languages technologies platforms is your web don't we have web driver 
"""
def encode(t):
    t = re.sub('[^A-Za-z0-9]', ' ', t ) 
    t = ' '.join([w for w in t.split(' ') if w not in sw])
    #t = ' '.join([ps.stem(w) for w in t.split(' ')])
    t = t.upper()
    t = re.sub('[AEIOUaeiou\s]', '', t)  
    return t
encode(t)

'HSLNGGSTCHNLGSPLTFRMSWBWBDRVR'

In [146]:
df['QHASH'] = df.QTEXT.apply(encode)
df

,ID,QGROUP,QPK,MAX_Q_PK,QOFFSET,MAX_QG_PK,QGOFFSET,MAX_FK_Form,QTEXT,QHASH
0,1-0-,2150,21200,23813,0,2617,0,2517,"Note:<BR />If ""No"" is selected for metric 1a, ...",NTBRFNSLCTDMTRC1MTRCS1B1C1D14KSTNBRFNSLCTDMTRC...
1,1-1-1a,2150,21201,23813,1,2617,0,2517,Has the head of the agency designated a Senior...,HSHDGNCYDSGNTDSNRGNCYFFCLPRVCYSPRQRDXCTVRDR137...
2,1-2-1b,2150,21202,23813,2,2617,0,2517,"Has the agency reported the name, title, and c...",HSGNCYRPRTDNMTTLCNTCTNFRMTNCRRNTSPMBMXWBSTFDRL...
3,1-3-1c,2150,21203,23813,3,2617,0,2517,"Does the SAOP have the necessary position, exp...",DSSPNCSSRYPSTNXPRTSTHRTYSRVRLSPSLCTPPLYNDCTNGN...
4,1-4-1d,2150,21204,23813,4,2617,0,2517,Does the SAOP have the necessary role and resp...,DSSPNCSSRYRLRSPNSBLTSWTHNGNCYFLLWNGSLCTPPLYNDC...
...,...,...,...,...,...,...,...,...,...,...
92,17-7-14g,2166,21402,23813,202,2617,16,2517,Does the agency's Privacy Program Page include...,DSGNCYPRVCYPRGRMPGNCLDLSTPRVDLNKSPBLCLYVLBLGNC...
93,17-9-14h,2166,21403,23813,203,2617,16,2517,Does the agency's Privacy Program Page include...,DSGNCYPRVCYPRGRMPGNCLDLSTPRVDLNKSPBLCLYVLBLGNC...
94,17-10-14i,2166,21404,23813,204,2617,16,2517,Does the agency's Privacy Program Page include...,DSGNCYPRVCYPRGRMPGNCLDNSTRCTNSCLRPLNLNGGNDVDLS...
95,17-11-14j,2166,21405,23813,205,2617,16,2517,Does the agency's Privacy Program Page include...,DSGNCYPRVCYPRGRMPGNCLDPPRPRTGNCYCNTCTNFRMTNNDV...
